### Library Imports

In [ ]:
# General Libraries
%matplotlib inline
import pandas as pd
import numpy as np
import os               # para manipulacion de directorios
import shutil           # para eliminar carpetas no vacias
!pip install mglearn    # el comando 'import mglearn' no funcionaba

# Preprocessing
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

# CountVectorizer | TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Cross Validation
from sklearn.model_selection import cross_val_score

# GridSearch
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# LogisticRegression Classifier
from sklearn.linear_model import LogisticRegression

# SVM Classifier
from sklearn import svm

# KNN Clasificator
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler

# Gaussian NaiveBayes Clasificator
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

# RandomForest
from sklearn.ensemble import RandomForestClassifier

# DecisionTree (Classifier and Regressor)
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

# Graphics library
import matplotlib.pyplot as plt

# MultiClass

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# MultiOutput
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, precision_recall_fscore_support
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
import pandas as pd

from sklearn.metrics import f1_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Read CSV
**Selección del la personalidad**

Se utiliza un valor dentro del rango de las personalidades para entrenar el clasificador binario en la personalidad seleccionada.
Los cuales tienen un total de 225 filas y cuentan con la siguiente estructura en columnas:

*   ID: Number
*   Personality: ['si', 'no']
*   Autodescripción: String

In [ ]:
# Load the Dataset

#path_colab  = 'https://drive.google.com/uc?id=1EXOwzmfqc0Wvlg7vQTu0Z6wG9pD15H2_'#"./_All_Classes_number.csv"
#path_colab  = 'https://drive.google.com/uc?id=1lt7U63P-tSD7fPgshEFMjw651qCGihys'#"./Augment_All_Classes_number.csv -> augment_gpt3_multilabel.csv # 227 texts
#path_colab   = 'https://drive.google.com/uc?id=1YYQXZE7LWVzalpit3auAWCKBnrddGCbO'# augment_gpt3_multilabel_fix.csv #192 texts
#path_colab  = 'https://drive.google.com/uc?id=1FFyWdFr-qpY2AsjRG8XoMo-JkcyxmsLg'#"./augment_gpt4_multilabel_RespPerc.csv" # 327 texts
path_colab  = 'https://drive.google.com/uc?id=1m8m8pIv3RN0zCvG3QdSOc1qxE-5SKGF6'#"./augment_gpt4_multilabel_RespPerc_fix.csv" # 297 texts


# With ONLY URL
if path_colab.startswith('http'):
  print("Ubication WEB")
  #path_colab='https://drive.google.com/uc?id=' + path_colab.split('/')[-2]
else: print("Ubication Local")


Ubication WEB


In [ ]:
# Importar un CSV con pandas a un formato DataFrame (df)

df = pd.read_csv(
    path_colab,
    encoding = "utf-8",
    header = 0
)

# Obtener TEXTOS
texts_unprocessed = list(df["Autodescripción"])

# Obtener PERSONALIDADES
df_all_classes = df.drop(['ID', 'Autodescripción'], axis=1)
#print(df_all_classes)
counts = []
categories = list(df_all_classes.columns.values)
for i in categories:
    counts.append((i, df_all_classes[i].sum()))

df.head()

,ID,AgradPerc,ApertPerc,ExtravPerc,NeurPerc,RespPerc,Autodescripción
0,15,0,1,1,1,0,me considero con una personalidad fuerte; norm...
1,35,0,0,0,0,0,Soy una persona por lo generalmente muy alegre...
2,54,0,1,0,1,1,Soy una persona confiable; que se encuentra pa...
3,123,0,0,0,0,0,Yo creo que mi personalidad o mi forma de ser;...
4,135,0,0,0,1,0,Me concidero una chica amable pero no muy soci...


In [ ]:
# obtener el número total de filas
num_filas = df.shape[0]
print(f"Total de textos: {num_filas}")

# obtener el numero total de positivos y negativos de cada clase
pos_AgradPerc = df['AgradPerc'].sum()
pos_ApertPerc = df['ApertPerc'].sum()
pos_ExtravPerc = df['ExtravPerc'].sum()
pos_NeurPerc = df['NeurPerc'].sum()
pos_RespPerc = df['RespPerc'].sum()

print(f'''
AgradPerc:  +({pos_AgradPerc}) -({num_filas-pos_AgradPerc})
ApertPerc:  +({pos_ApertPerc}) -({num_filas-pos_ApertPerc})
ExtravPerc: +({pos_ExtravPerc}) -({num_filas-pos_ExtravPerc})
NeurPerc:   +({pos_NeurPerc}) -({num_filas-pos_NeurPerc})
RespPerc:   +({pos_RespPerc}) -({num_filas-pos_RespPerc})
''')

''' Before
Total de textos: 227
AgradPerc:  +(173) -(54)
ApertPerc:  +(144) -(83)
ExtravPerc: +(60) -(167)
NeurPerc:   +(102) -(125)
RespPerc:   +(124) -(103) '''

Total de textos: 297
      
AgradPerc:  +(143) -(154) 
ApertPerc:  +(144) -(153) 
ExtravPerc: +(70) -(227) 
NeurPerc:   +(152) -(145) 
RespPerc:   +(114) -(183) 



' Before\nTotal de textos: 227\nAgradPerc:  +(173) -(54) \nApertPerc:  +(144) -(83) \nExtravPerc: +(60) -(167) \nNeurPerc:   +(102) -(125) \nRespPerc:   +(124) -(103) '

### Preprocesamiento
**Preprocesamiento del texto (opcional)**

Es posible realizar una limpieza del texto, quitando carácteres innecesarios como números, símbolos, e incluso palabras que no aportan significancia a la oración.

Antes de aplicar la funcion CountVectorizer o TfidfVectorizer (es decir, obtener la representación), es posible realizar un preprocesamiento del texto de entrada para ´limpiarlo´ un poco.

En este caso, luego de leer el corpus se procede a preprocesar el texto de la columna Autodescripción con las siguientes técnicas.

In [ ]:
def preprocessing(texto):
    texto = re.sub(r'[^\w ]', "", texto)      # Eliminar todo lo que no sea una palabra
    texto = texto.lower()                     # Pasa el texto a minúscula
    letras_dobles = "abdfghijkmnñpqstuvwxyz"  # Remover letras con 2 ocurrencias (con excepciones). Excepciones: ee-cc-ll-rr-oo (y mayus)
    letras_dobles += letras_dobles.upper()
    texto = re.sub("(?P<char>[" + re.escape(letras_dobles) + "])(?P=char)+", r"\1", texto)
    texto = re.sub(r'([\w\W])\1{2,}', r'\1', texto)  # remover caracteres que se repiten al menos 3 veces
    return texto

texts_processed = []

for i in texts_unprocessed:
    x = preprocessing(i)
    texts_processed.append(x)

print("Muestra procesada(size=%d): %s"       %( len(texts_processed[0]), texts_processed[0]) )
print("\nMuestra sin procesar(size=%d): %s"  %( len(texts_unprocessed[0]), texts_unprocessed[0]) )


# Selección de Texto original o preprocesado (Comentar el que no se utiliza)

# texts = texts_unprocessed
texts = texts_processed

Muestra procesada(size=3838): me considero con una personalidad fuerte normalmente me muestro bastante mala con la gente que no conozco pero con mis amigos soy alguien distinto soy medio distante no soy de ser muy cariñosa siempre depende la otra persona siempre me considere muy divertida me encanta hacer reír a los demás soy muy deportista me gusta estar en movimiento desde los 4 años que hago deporte hice danza y también salgo a trotar o andar en rollers mis metas en la vida son recibirme viajar y tener 2 hijos hoy en día me siento bien años anteriores no la estaba pasando muy bien pero empece a ver el lado lindo de la vida gracias a mis amigos y mi familia aunque soy muy desapegada la familia y amigos son algo muy importante en mi vida actualmente vivo con mi tía que es como mi mamá ella es la persona más importante y después está papá gracias a ella estoy donde estoy y estudiando lo que me gusta siento que el por qué me gusta la psicología forma parte de mi personalidad siempre me 

In [ ]:
# Utilizar stop_words extendido

# stopwords_sp_web = ["algún","alguna","algunas","alguno","algunos","ambos","ampleamos","ante","antes","aquel","aquellas","aquellos","aqui","arriba","atras","bajo","bastante","bien","cada","cierta","ciertas","cierto","ciertos","como","con","conseguimos","conseguir","consigo","consigue","consiguen","consigues","cual","cuando","dentro","desde","donde","dos","el","ellas","ellos","empleais","emplean","emplear","empleas","empleo","en","encima","entonces","entre","era","eramos","eran","eras","eres","es","esta","estaba","estado","estais","estamos","estan","estoy","fin","fue","fueron","fui","fuimos","gueno","ha","hace","haceis","hacemos","hacen","hacer","haces","hago","incluso","intenta","intentais","intentamos","intentan","intentar","intentas","intento","ir","la","largo","las","lo","los","mientras","mio","modo","muchos","muy","nos","nosotros","otro","para","pero","podeis","podemos","poder","podria","podriais","podriamos","podrian","podrias","por","por qué","porque","primero","puede","pueden","puedo","quien","sabe","sabeis","sabemos","saben","saber","sabes","ser","si","siendo","sin","sobre","sois","solamente","solo","somos","soy","su","sus","también","teneis","tenemos","tener","tengo","tiempo","tiene","tienen","todo","trabaja","trabajais","trabajamos","trabajan","trabajar","trabajas","trabajo","tras","tuyo","ultimo","un","una","unas","uno","unos","usa","usais","usamos","usan","usar","usas","uso","va","vais","valor","vamos","van","vaya","verdad","verdadera","verdadero","vosotras","vosotros","voy","yo","él","ésta","éstas","éste","éstos","última","últimas","último","últimos","a","añadió","aún","actualmente","adelante","además","afirmó","agregó","ahí","ahora","al","algo","alrededor","anterior","apenas","aproximadamente","aquí","así","aseguró","aunque","ayer","buen","buena","buenas","bueno","buenos","cómo","casi","cerca","cinco","comentó","conocer","consideró","considera","contra","cosas","creo","cuales","cualquier","cuanto","cuatro","cuenta","da","dado","dan","dar","de","debe","deben","debido","decir","dejó","del","demás","después","dice","dicen","dicho","dieron","diferente","diferentes","dijeron","dijo","dio","durante","e","ejemplo","ella","ello","embargo","encuentra","esa","esas","ese","eso","esos","está","están","estaban","estar","estará","estas","este","esto","estos","estuvo","ex","existe","existen","explicó","expresó","fuera","gran","grandes","había","habían","haber","habrá","hacerlo","hacia","haciendo","han","hasta","hay","haya","he","hecho","hemos","hicieron","hizo","hoy","hubo","igual","indicó","informó","junto","lado","le","les","llegó","lleva","llevar","luego","lugar","más","manera","manifestó","mayor","me","mediante","mejor","mencionó","menos","mi","misma","mismas","mismo","mismos","momento","mucha","muchas","mucho","nada","nadie","ni","ningún","ninguna","ningunas","ninguno","ningunos","no","nosotras","nuestra","nuestras","nuestro","nuestros","nueva","nuevas","nuevo","nuevos","nunca","o","ocho","otra","otras","otros","parece","parte","partir","pasada","pasado","pesar","poca","pocas","poco","pocos","podrá","podrán","podría","podrían","poner","posible","próximo","próximos","primer","primera","primeros","principalmente","propia","propias","propio","propios","pudo","pueda","pues","qué","que","quedó","queremos","quién","quienes","quiere","realizó","realizado","realizar","respecto","sí","sólo","se","señaló","sea","sean","según","segunda","segundo","seis","será","serán","sería","sido","siempre","siete","sigue","siguiente","sino","sola","solas","solos","son","tal","tampoco","tan","tanto","tenía","tendrá","tendrán","tenga","tenido","tercera","toda","todas","todavía","todos","total","trata","través","tres","tuvo","usted","varias","varios","veces","ver","vez","y","ya"]
stopwords_sp_web_accent_less = set({"algun","alguna","algunas","alguno","algunos","ambos","ampleamos","ante","antes","aquel","aquellas","aquellos","aqui","arriba","atras","bajo","bastante","bien","cada","cierta","ciertas","cierto","ciertos","como","con","conseguimos","conseguir","consigo","consigue","consiguen","consigues","cual","cuando","dentro","desde","donde","dos","el","ellas","ellos","empleais","emplean","emplear","empleas","empleo","en","encima","entonces","entre","era","eramos","eran","eras","eres","es","esta","estaba","estado","estais","estamos","estan","estoy","fin","fue","fueron","fui","fuimos","gueno","ha","hace","haceis","hacemos","hacen","hacer","haces","hago","incluso","intenta","intentais","intentamos","intentan","intentar","intentas","intento","ir","la","largo","las","lo","los","mientras","mio","modo","muchos","muy","nos","nosotros","otro","para","pero","podeis","podemos","poder","podria","podriais","podriamos","podrian","podrias","por","por que","porque","primero","puede","pueden","puedo","quien","sabe","sabeis","sabemos","saben","saber","sabes","ser","si","siendo","sin","sobre","sois","solamente","solo","somos","soy","su","sus","tambien","teneis","tenemos","tener","tengo","tiempo","tiene","tienen","todo","trabaja","trabajais","trabajamos","trabajan","trabajar","trabajas","trabajo","tras","tuyo","ultimo","un","una","unas","uno","unos","usa","usais","usamos","usan","usar","usas","uso","va","vais","valor","vamos","van","vaya","verdad","verdadera","verdadero","vosotras","vosotros","voy","yo","el","esta","estas","este","estos","ultima","ultimas","ultimo","ultimos","a","anadio","aun","actualmente","adelante","ademas","afirmo","agrego","ahi","ahora","al","algo","alrededor","anterior","apenas","aproximadamente","aqui","asi","aseguro","aunque","ayer","buen","buena","buenas","bueno","buenos","como","casi","cerca","cinco","comento","conocer","considero","considera","contra","cosas","creo","cuales","cualquier","cuanto","cuatro","cuenta","da","dado","dan","dar","de","debe","deben","debido","decir","dejo","del","demas","despues","dice","dicen","dicho","dieron","diferente","diferentes","dijeron","dijo","dio","durante","e","ejemplo","ella","ello","embargo","encuentra","esa","esas","ese","eso","esos","esta","estan","estaban","estar","estara","estas","este","esto","estos","estuvo","ex","existe","existen","explico","expreso","fuera","gran","grandes","habia","habian","haber","habra","hacerlo","hacia","haciendo","han","hasta","hay","haya","he","hecho","hemos","hicieron","hizo","hoy","hubo","igual","indico","informo","junto","lado","le","les","llego","lleva","llevar","luego","lugar","mas","manera","manifesto","mayor","me","mediante","mejor","menciono","menos","mi","misma","mismas","mismo","mismos","momento","mucha","muchas","mucho","nada","nadie","ni","ningun","ninguna","ningunas","ninguno","ningunos","no","nosotras","nuestra","nuestras","nuestro","nuestros","nueva","nuevas","nuevo","nuevos","nunca","o","ocho","otra","otras","otros","parece","parte","partir","pasada","pasado","pesar","poca","pocas","poco","pocos","podra","podran","podria","podrian","poner","posible","proximo","proximos","primer","primera","primeros","principalmente","propia","propias","propio","propios","pudo","pueda","pues","que","que","quedo","queremos","quien","quienes","quiere","realizo","realizado","realizar","respecto","si","solo","se","senalo","sea","sean","segun","segunda","segundo","seis","sera","seran","seria","sido","siempre","siete","sigue","siguiente","sino","sola","solas","solos","son","tal","tampoco","tan","tanto","tenia","tendra","tendran","tenga","tenido","tercera","toda","todas","todavia","todos","total","trata","traves","tres","tuvo","usted","varias","varios","veces","ver","vez","y","ya"})
stopwords_sp = list(stopwords_sp_web_accent_less)


In [ ]:
vect_tf = TfidfVectorizer(
    use_idf = False,            # default=True
    lowercase = True,           # Lowercase: default=True
    strip_accents = 'unicode',  # Remover acentos
    stop_words = stopwords_sp,  # Palabras para no tener en cuenta ya que no agregan mucho significado a una oracion.
    max_features = 500,         # maximo de vocabulario
    max_df = 0.8,               # (frecuencia máxima de documentos) es un valor que representa el porcentaje máximo de documentos en los que un término puede aparecer para ser incluido en el vocabulario. Los términos que aparecen en más del max_df% de los documentos se consideran comunes y, por lo tanto, no son relevantes para el análisis.
    min_df = 2,                 # (frecuencia mínima de documentos) es un valor entero que representa la frecuencia mínima con la que un término debe aparecer en los documentos para ser incluido en el vocabulario. Los términos que aparecen en menos de min_df documentos no se consideran relevantes para el análisis y se omiten del vocabulario.
    analyzer = 'word',
    norm=None
    #ngram_range=(2,6)
)

vect_tfidf = TfidfVectorizer(
    use_idf = True,             # default=True
    lowercase = True,           # Lowercase: default=True
    strip_accents = 'unicode',  # Remover acentos
    stop_words = stopwords_sp,  # Palabras para no tener en cuenta ya que no agregan mucho significado a una oracion.
    max_features = 500,         # maximo de vocabulario
    max_df = 0.8,               # (frecuencia máxima de documentos) es un valor que representa el porcentaje máximo de documentos en los que un término puede aparecer para ser incluido en el vocabulario. Los términos que aparecen en más del max_df% de los documentos se consideran comunes y, por lo tanto, no son relevantes para el análisis.
    min_df = 2,                 # (frecuencia mínima de documentos) es un valor entero que representa la frecuencia mínima con la que un término debe aparecer en los documentos para ser incluido en el vocabulario. Los términos que aparecen en menos de min_df documentos no se consideran relevantes para el análisis y se omiten del vocabulario.
    analyzer = 'word',
    norm=None
    #ngram_range=(2,6)
)

vect_binary = CountVectorizer(
    binary = True,              # default=True
    lowercase = True,           # Lowercase: default=True
    strip_accents = 'unicode',  # Remover acentos
    stop_words = stopwords_sp,  # Palabras para no tener en cuenta ya que no agregan mucho significado a una oracion.
    max_features = 500,         # maximo de vocabulario
    max_df = 0.8,               # (frecuencia máxima de documentos) es un valor que representa el porcentaje máximo de documentos en los que un término puede aparecer para ser incluido en el vocabulario. Los términos que aparecen en más del max_df% de los documentos se consideran comunes y, por lo tanto, no son relevantes para el análisis.
    min_df = 2,                 # (frecuencia mínima de documentos) es un valor entero que representa la frecuencia mínima con la que un término debe aparecer en los documentos para ser incluido en el vocabulario. Los términos que aparecen en menos de min_df documentos no se consideran relevantes para el análisis y se omiten del vocabulario.
    analyzer = 'word',
    #ngram_range=(2,6),
)

vect_count = CountVectorizer(
    binary = False,             # default=True
    lowercase = True,           # Lowercase: default=True
    strip_accents = 'unicode',  # Remover acentos
    stop_words = stopwords_sp,  # Palabras para no tener en cuenta ya que no agregan mucho significado a una oracion.
    max_features = 500,         # maximo de vocabulario
    max_df = 0.8,               # (frecuencia máxima de documentos) es un valor que representa el porcentaje máximo de documentos en los que un término puede aparecer para ser incluido en el vocabulario. Los términos que aparecen en más del max_df% de los documentos se consideran comunes y, por lo tanto, no son relevantes para el análisis.
    min_df = 2,                 # (frecuencia mínima de documentos) es un valor entero que representa la frecuencia mínima con la que un término debe aparecer en los documentos para ser incluido en el vocabulario. Los términos que aparecen en menos de min_df documentos no se consideran relevantes para el análisis y se omiten del vocabulario.
    analyzer = 'word',
    #ngram_range=(2,6),
)


In [ ]:
# Select the Representation

# 1: TF           # (float) Frecuencia de términos mide la frecuencia con la que aparece un término en un documento.
# 0: TFIDF        # (float) TF por la IDF (Aumenta el peso de los terminos menos frecuentes y disminuye el peso de los terminos mas frecuentes)
# 2: BINARY       # (int) Coloca 1 o 0 si la palabra está contenida en el documento.
# 3: COUNT        # (int) Coloca el total de palabras que aparecen en un documento.

representations = [ vect_tf, vect_tfidf, vect_binary, vect_count ]
represent_names = [ 'TF', 'TFIDF', 'BINARY', 'COUNT' ]

REP = 2

vect    = representations[REP]
X_train = vect.fit_transform(texts) # Return: sparse matrix of (n_samples, n_features) Tf-idf-weighted document-term matrix.

y = df[categories] # Se puede realizar de la siguiente manera tambien: df[['AgradPerc', 'ApertPerc', 'ExtravPerc', 'NeurPerc', 'RespPerc']]


### Results

####MultinomialNB

In [ ]:
# MultinomialNB

NB_pipeline = Pipeline([
                ('tfidf', vect ),
                ('clf', MultiOutputClassifier(MultinomialNB( fit_prior=True, class_prior=None  ))),
            ])


#scores = cross_val_score(NB_pipeline, texts, y, cv=5, scoring='f1_weighted')
scores = cross_val_score(NB_pipeline, texts, y, cv=KFold(n_splits=5,shuffle=True, random_state=1), scoring='f1_weighted' )

#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("CV Accuracy Scores:\n", scores)
print("Mean CV Accuracy: ", round(scores.mean(), 4))
print("Standard Deviation: ", scores.std() * 2)

''' Without Augment
CV Accuracy Scores:
 [0.4233871  0.45818976 0.63650633 0.57364571 0.70289601]
Mean CV Accuracy:  0.5589
Standard Deviation:  0.21067762180105937 '''

''' With Augment v1
CV Accuracy Scores:
 [0.46858712 0.27240562 0.39256462 0.91675471 0.62596478]
Mean CV Accuracy:  0.5353
Standard Deviation:  0.44512042808733066 '''

''' With Augment v2
CV Accuracy Scores:
 [0.54008722 0.30036707 0.34858622 0.45204026 0.53869275]
Mean CV Accuracy:  0.436
Standard Deviation:  0.19529401683978515

augment_gpt3_multilabel
Mean CV Accuracy:  0.6718

augment_gpt3_fix
Mean CV Accuracy:  0.6546

augment_gpt4_multilabel_RespPerc.csv
Mean CV Accuracy:  0.646

augment_gpt4_multilabel_RespPerc_fix.csv
Mean CV Accuracy:  0.6055
'''

CV Accuracy Scores:
 [0.57811075 0.62108373 0.63770941 0.60973748 0.54935405]
Mean CV Accuracy:  0.5992
Standard Deviation:  0.06326000972681407


' With Augment v2\nCV Accuracy Scores:\n [0.54008722 0.30036707 0.34858622 0.45204026 0.53869275]\nMean CV Accuracy:  0.436\nStandard Deviation:  0.19529401683978515 \n\naugment_gpt3_multilabel\nMean CV Accuracy:  0.6718\n\naugment_gpt3_fix\nMean CV Accuracy:  0.6546\n\naugment_gpt4_multilabel_RespPerc.csv\nMean CV Accuracy:  0.646\n\naugment_gpt4_multilabel_RespPerc_fix.csv\nMean CV Accuracy:  0.6055\n'

In [ ]:
# Realizamos validación cruzada y obtenemos las predicciones
#predicciones = cross_val_predict(NB_pipeline, texts, y, cv=5)
predicciones = cross_val_predict(NB_pipeline, texts, y, cv=KFold(n_splits=5,shuffle=True, random_state=1))

# Calculamos el f1_weighted para cada columna de salida y el f1_weighted medio ponderado
f1_scores = []
for i, columna in enumerate(y.columns):
    f1 = f1_score(y[columna], predicciones[:, i], average='weighted')
    f1_scores.append(f1)
    print(f"f1_weighted para {columna}: {f1:.4f}")

f1_weighted_media = sum(f1_scores) / len(f1_scores)
print(f"f1_weighted medio: {f1_weighted_media:.4f}")

''' Without Augment
f1_weighted para AgradPerc: 0.4763
f1_weighted para ApertPerc: 0.5827
f1_weighted para ExtravPerc: 0.6431
f1_weighted para NeurPerc: 0.5907
f1_weighted para RespPerc: 0.6906
f1_weighted medio: 0.5967 '''

''' With Augment v1
f1_weighted para AgradPerc: 0.4306
f1_weighted para ApertPerc: 0.5628
f1_weighted para ExtravPerc: 0.5370
f1_weighted para NeurPerc: 0.5576
f1_weighted para RespPerc: 0.8354
f1_weighted medio: 0.5847 '''

''' With Augment v2
f1_weighted para AgradPerc: 0.3240
f1_weighted para ApertPerc: 0.5319
f1_weighted para ExtravPerc: 0.6943
f1_weighted para NeurPerc: 0.6133
f1_weighted para RespPerc: 0.5859
f1_weighted medio: 0.5499

augment_gpt3_multilabel
f1_weighted para AgradPerc: 0.6378
f1_weighted para ApertPerc: 0.5803
f1_weighted para ExtravPerc: 0.6668
f1_weighted para NeurPerc: 0.5639
f1_weighted para RespPerc: 0.8585
f1_weighted medio: 0.6615

augment_gpt3_fix
f1_weighted para AgradPerc: 0.6799
f1_weighted para ApertPerc: 0.5257
f1_weighted para ExtravPerc: 0.6183
f1_weighted para NeurPerc: 0.5674
f1_weighted para RespPerc: 0.8194
f1_weighted medio: 0.6421

augment_gpt4_multilabel_RespPerc.csv
f1_weighted para AgradPerc: 0.6883
f1_weighted para ApertPerc: 0.6487
f1_weighted para ExtravPerc: 0.7114
f1_weighted para NeurPerc: 0.6365
f1_weighted para RespPerc: 0.7105
f1_weighted medio: 0.6791

augment_gpt4_multilabel_RespPerc_fix.csv
f1_weighted para AgradPerc: 0.6557
f1_weighted para ApertPerc: 0.6060
f1_weighted para ExtravPerc: 0.7051
f1_weighted para NeurPerc: 0.6492
f1_weighted para RespPerc: 0.6810
f1_weighted medio: 0.6594
'''

f1_weighted para AgradPerc: 0.6687
f1_weighted para ApertPerc: 0.6261
f1_weighted para ExtravPerc: 0.6839
f1_weighted para NeurPerc: 0.6252
f1_weighted para RespPerc: 0.6712
f1_weighted medio: 0.6550


' With Augment v2\nf1_weighted para AgradPerc: 0.3240\nf1_weighted para ApertPerc: 0.5319\nf1_weighted para ExtravPerc: 0.6943\nf1_weighted para NeurPerc: 0.6133\nf1_weighted para RespPerc: 0.5859\nf1_weighted medio: 0.5499 \n\naugment_gpt3_multilabel\nf1_weighted para AgradPerc: 0.6378\nf1_weighted para ApertPerc: 0.5803\nf1_weighted para ExtravPerc: 0.6668\nf1_weighted para NeurPerc: 0.5639\nf1_weighted para RespPerc: 0.8585\nf1_weighted medio: 0.6615\n\naugment_gpt3_fix\nf1_weighted para AgradPerc: 0.6799\nf1_weighted para ApertPerc: 0.5257\nf1_weighted para ExtravPerc: 0.6183\nf1_weighted para NeurPerc: 0.5674\nf1_weighted para RespPerc: 0.8194\nf1_weighted medio: 0.6421\n\naugment_gpt4_multilabel_RespPerc.csv\nf1_weighted para AgradPerc: 0.6883\nf1_weighted para ApertPerc: 0.6487\nf1_weighted para ExtravPerc: 0.7114\nf1_weighted para NeurPerc: 0.6365\nf1_weighted para RespPerc: 0.7105\nf1_weighted medio: 0.6791\n\naugment_gpt4_multilabel_RespPerc_fix.csv\nf1_weighted para AgradPer

####SVC_pipeline

In [ ]:
# SVC
SVC_pipeline = Pipeline([
                ('tfidf', vect ),
                ('clf', MultiOutputClassifier(svm.SVC(random_state=1, class_weight='balanced'), n_jobs=1)),
            ])

#scores = cross_val_score(SVC_pipeline, texts, y, cv=5, scoring='f1_weighted')
scores = cross_val_score(SVC_pipeline, texts, y, cv=KFold(n_splits=5,shuffle=True, random_state=1), scoring='f1_weighted' )

#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("CV Accuracy Scores:\n", scores)
print("Mean CV Accuracy: ", round(scores.mean(), 4))
print("Standard Deviation: ", scores.std() * 2)

''' Without Augment
CV Accuracy Scores:
 [0.48860867 0.4608431  0.60044444 0.52831558 0.6391725 ]
Mean CV Accuracy:  0.5435
Standard Deviation:  0.13407962374128868 '''

''' With Augment v1
CV Accuracy Scores:
 [0.55075414 0.35498309 0.49354399 0.92928386 0.64544569]
Mean CV Accuracy:  0.5948
Standard Deviation:  0.38382225243583024 '''

''' With Augment v2
CV Accuracy Scores:
 [0.48033538 0.22054698 0.4483185  0.58703983 0.67637448]
Mean CV Accuracy:  0.4825
Standard Deviation:  0.30764698041928257

augment_gpt3_multilabel
Mean CV Accuracy:  0.6999

augment_gpt3_fix
Mean CV Accuracy:  0.6388

augment_gpt4_multilabel_RespPerc.csv
Mean CV Accuracy:  0.6848

augment_gpt4_multilabel_RespPerc_fix.csv
Mean CV Accuracy:  0.6365

'''

CV Accuracy Scores:
 [0.60088058 0.64908316 0.6659512  0.67064581 0.56967489]
Mean CV Accuracy:  0.6312
Standard Deviation:  0.07893298453956053


' With Augment v2\nCV Accuracy Scores:\n [0.48033538 0.22054698 0.4483185  0.58703983 0.67637448]\nMean CV Accuracy:  0.4825\nStandard Deviation:  0.30764698041928257 \n\naugment_gpt3_multilabel\nMean CV Accuracy:  0.6999\n\naugment_gpt3_fix\nMean CV Accuracy:  0.6388\n\naugment_gpt4_multilabel_RespPerc.csv\nMean CV Accuracy:  0.6848\n\naugment_gpt4_multilabel_RespPerc_fix.csv\nMean CV Accuracy:  0.6365\n\n'

In [ ]:
# Realizamos validación cruzada y obtenemos las predicciones
#predicciones = cross_val_predict(SVC_pipeline, texts, y, cv=5)
predicciones = cross_val_predict(SVC_pipeline, texts, y, cv=KFold(n_splits=5,shuffle=True, random_state=1))

# Calculamos el f1_weighted para cada columna de salida y el f1_weighted medio ponderado
f1_scores = []
for i, columna in enumerate(y.columns):
    f1 = f1_score(y[columna], predicciones[:, i], average='weighted')
    f1_scores.append(f1)
    print(f"f1_weighted para {columna}: {f1:.4f}")

f1_weighted_media = sum(f1_scores) / len(f1_scores)
print(f"f1_weighted medio: {f1_weighted_media:.4f}")

''' Without Augment
f1_weighted para AgradPerc: 0.4663
f1_weighted para ApertPerc: 0.5708
f1_weighted para ExtravPerc: 0.5614
f1_weighted para NeurPerc: 0.4894
f1_weighted para RespPerc: 0.6840
f1_weighted medio: 0.5544
'''

''' With Augment v1
f1_weighted para AgradPerc: 0.4482
f1_weighted para ApertPerc: 0.5786
f1_weighted para ExtravPerc: 0.5739
f1_weighted para NeurPerc: 0.6099
f1_weighted para RespPerc: 0.8186
f1_weighted medio: 0.6058
'''
''' With Augment v2
f1_weighted para AgradPerc: 0.3741
f1_weighted para ApertPerc: 0.6224
f1_weighted para ExtravPerc: 0.7651
f1_weighted para NeurPerc: 0.6175
f1_weighted para RespPerc: 0.6558
f1_weighted medio: 0.6070

augment_gpt3_multilabel
f1_weighted para AgradPerc: 0.6621
f1_weighted para ApertPerc: 0.5862
f1_weighted para ExtravPerc: 0.6473
f1_weighted para NeurPerc: 0.6034
f1_weighted para RespPerc: 0.8410
f1_weighted medio: 0.6680

augment_gpt3_fix
f1_weighted para AgradPerc: 0.6315
f1_weighted para ApertPerc: 0.5323
f1_weighted para ExtravPerc: 0.5970
f1_weighted para NeurPerc: 0.5387
f1_weighted para RespPerc: 0.8024
f1_weighted medio: 0.6204

augment_gpt4_multilabel_RespPerc.csv
f1_weighted para AgradPerc: 0.7094
f1_weighted para ApertPerc: 0.6874
f1_weighted para ExtravPerc: 0.7650
f1_weighted para NeurPerc: 0.7086
f1_weighted para RespPerc: 0.7718
f1_weighted medio: 0.7284

augment_gpt4_multilabel_RespPerc_fix.csv
f1_weighted para AgradPerc: 0.6762
f1_weighted para ApertPerc: 0.6563
f1_weighted para ExtravPerc: 0.7218
f1_weighted para NeurPerc: 0.6933
f1_weighted para RespPerc: 0.7725
f1_weighted medio: 0.7040
'''

f1_weighted para AgradPerc: 0.7072
f1_weighted para ApertPerc: 0.6492
f1_weighted para ExtravPerc: 0.6987
f1_weighted para NeurPerc: 0.6934
f1_weighted para RespPerc: 0.7879
f1_weighted medio: 0.7073


' With Augment v2\nf1_weighted para AgradPerc: 0.3741\nf1_weighted para ApertPerc: 0.6224\nf1_weighted para ExtravPerc: 0.7651\nf1_weighted para NeurPerc: 0.6175\nf1_weighted para RespPerc: 0.6558\nf1_weighted medio: 0.6070\n\naugment_gpt3_multilabel\nf1_weighted para AgradPerc: 0.6621\nf1_weighted para ApertPerc: 0.5862\nf1_weighted para ExtravPerc: 0.6473\nf1_weighted para NeurPerc: 0.6034\nf1_weighted para RespPerc: 0.8410\nf1_weighted medio: 0.6680\n\naugment_gpt3_fix\nf1_weighted para AgradPerc: 0.6315\nf1_weighted para ApertPerc: 0.5323\nf1_weighted para ExtravPerc: 0.5970\nf1_weighted para NeurPerc: 0.5387\nf1_weighted para RespPerc: 0.8024\nf1_weighted medio: 0.6204\n\naugment_gpt4_multilabel_RespPerc.csv\nf1_weighted para AgradPerc: 0.7094\nf1_weighted para ApertPerc: 0.6874\nf1_weighted para ExtravPerc: 0.7650\nf1_weighted para NeurPerc: 0.7086\nf1_weighted para RespPerc: 0.7718\nf1_weighted medio: 0.7284\n\naugment_gpt4_multilabel_RespPerc_fix.csv\nf1_weighted para AgradPerc

####DecissionTreeClassifier

In [ ]:
# DecissionTreeClassifier

DTC_pipeline = Pipeline([
                ('tfidf', vect ),
                ('clf', MultiOutputClassifier(DecisionTreeClassifier(
                    criterion='gini',
                    max_depth= 5,
                    min_samples_leaf= 11,
                    splitter='best',
                    random_state=1), n_jobs=1)),
            ])


#scores = cross_val_score(DTC_pipeline, texts, y, cv=5, scoring='f1_weighted')
scores = cross_val_score(DTC_pipeline, texts, y, cv=KFold(n_splits=5,shuffle=True, random_state=1), scoring='f1_weighted' )

#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("CV Accuracy Scores:\n", scores)
print("Mean CV Accuracy: ", round(scores.mean(), 4))
print("Standard Deviation: ", scores.std() * 2)

''' Without Augment
CV Accuracy Scores:
 [0.41465517 0.45264414 0.53197133 0.64081678 0.61868626]
Mean CV Accuracy:  0.5318
Standard Deviation:  0.17758932376221692
'''
''' With Augment v1
CV Accuracy Scores:
 [0.54450099 0.53974068 0.5883676  0.86711205 0.65198172]
Mean CV Accuracy:  0.6383
Standard Deviation:  0.24256557918226068
'''
''' With Augment v2
CV Accuracy Scores:
 [0.47272454 0.35883433 0.54863933 0.43092006 0.55499263]
Mean CV Accuracy:  0.4732
Standard Deviation:  0.14764403084225147

augment_gpt3_multilabel
Mean CV Accuracy:  0.6422

augment_gpt3_fix
Mean CV Accuracy:  0.583

augment_gpt4_multilabel_RespPerc.csv
Mean CV Accuracy:  0.5778

augment_gpt4_multilabel_RespPerc_fix.csv
Mean CV Accuracy:  0.5106
'''

CV Accuracy Scores:
 [0.4936805  0.5442077  0.63164201 0.5402968  0.45755964]
Mean CV Accuracy:  0.5335
Standard Deviation:  0.11711612564721742


' With Augment v2\nCV Accuracy Scores:\n [0.47272454 0.35883433 0.54863933 0.43092006 0.55499263]\nMean CV Accuracy:  0.4732\nStandard Deviation:  0.14764403084225147\n\naugment_gpt3_multilabel\nMean CV Accuracy:  0.6422\n\naugment_gpt3_fix\nMean CV Accuracy:  0.583\n\naugment_gpt4_multilabel_RespPerc.csv\nMean CV Accuracy:  0.5778\n\naugment_gpt4_multilabel_RespPerc_fix.csv\nMean CV Accuracy:  0.5106\n'

In [ ]:
# Realizamos validación cruzada y obtenemos las predicciones
#predicciones = cross_val_predict(DTC_pipeline, texts, y, cv=5)
predicciones = cross_val_predict(DTC_pipeline, texts, y, cv=KFold(n_splits=5,shuffle=True, random_state=1))

# Calculamos el f1_weighted para cada columna de salida y el f1_weighted medio ponderado
f1_scores = []
for i, columna in enumerate(y.columns):
    f1 = f1_score(y[columna], predicciones[:, i], average='weighted')
    f1_scores.append(f1)
    print(f"f1_weighted para {columna}: {f1:.4f}")

f1_weighted_media = sum(f1_scores) / len(f1_scores)
print(f"f1_weighted medio: {f1_weighted_media:.4f}")

''' Without Augment
f1_weighted para AgradPerc: 0.5517
f1_weighted para ApertPerc: 0.5688
f1_weighted para ExtravPerc: 0.5453
f1_weighted para NeurPerc: 0.5303
f1_weighted para RespPerc: 0.6891
f1_weighted medio: 0.5771
'''
''' With Augment v1
f1_weighted para AgradPerc: 0.5587
f1_weighted para ApertPerc: 0.5312
f1_weighted para ExtravPerc: 0.6098
f1_weighted para NeurPerc: 0.5832
f1_weighted para RespPerc: 0.7274
f1_weighted medio: 0.6020
'''
''' With Augment v2
f1_weighted para AgradPerc: 0.3396
f1_weighted para ApertPerc: 0.6454
f1_weighted para ExtravPerc: 0.6785
f1_weighted para NeurPerc: 0.6336
f1_weighted para RespPerc: 0.5207
f1_weighted medio: 0.5636

augment_gpt3_multilabel
f1_weighted para AgradPerc: 0.6388
f1_weighted para ApertPerc: 0.5468
f1_weighted para ExtravPerc: 0.6062
f1_weighted para NeurPerc: 0.5686
f1_weighted para RespPerc: 0.7581
f1_weighted medio: 0.6237

augment_gpt3_fix
f1_weighted para AgradPerc: 0.6017
f1_weighted para ApertPerc: 0.4533
f1_weighted para ExtravPerc: 0.5513
f1_weighted para NeurPerc: 0.5417
f1_weighted para RespPerc: 0.7305
f1_weighted medio: 0.5757

augment_gpt4_multilabel_RespPerc.csv
f1_weighted para AgradPerc: 0.5966
f1_weighted para ApertPerc: 0.5957
f1_weighted para ExtravPerc: 0.6740
f1_weighted para NeurPerc: 0.6271
f1_weighted para RespPerc: 0.6638
f1_weighted medio: 0.6315

augment_gpt4_multilabel_RespPerc_fix.csv
f1_weighted para AgradPerc: 0.5990
f1_weighted para ApertPerc: 0.5249
f1_weighted para ExtravPerc: 0.6403
f1_weighted para NeurPerc: 0.5904
f1_weighted para RespPerc: 0.6455
f1_weighted medio: 0.6000
'''

f1_weighted para AgradPerc: 0.6059
f1_weighted para ApertPerc: 0.5852
f1_weighted para ExtravPerc: 0.6531
f1_weighted para NeurPerc: 0.5682
f1_weighted para RespPerc: 0.6696
f1_weighted medio: 0.6164


' With Augment v2\nf1_weighted para AgradPerc: 0.3396\nf1_weighted para ApertPerc: 0.6454\nf1_weighted para ExtravPerc: 0.6785\nf1_weighted para NeurPerc: 0.6336\nf1_weighted para RespPerc: 0.5207\nf1_weighted medio: 0.5636\n\naugment_gpt3_multilabel\nf1_weighted para AgradPerc: 0.6388\nf1_weighted para ApertPerc: 0.5468\nf1_weighted para ExtravPerc: 0.6062\nf1_weighted para NeurPerc: 0.5686\nf1_weighted para RespPerc: 0.7581\nf1_weighted medio: 0.6237\n\naugment_gpt3_fix\nf1_weighted para AgradPerc: 0.6017\nf1_weighted para ApertPerc: 0.4533\nf1_weighted para ExtravPerc: 0.5513\nf1_weighted para NeurPerc: 0.5417\nf1_weighted para RespPerc: 0.7305\nf1_weighted medio: 0.5757\n\naugment_gpt4_multilabel_RespPerc.csv\nf1_weighted para AgradPerc: 0.5966\nf1_weighted para ApertPerc: 0.5957\nf1_weighted para ExtravPerc: 0.6740\nf1_weighted para NeurPerc: 0.6271\nf1_weighted para RespPerc: 0.6638\nf1_weighted medio: 0.6315\n\naugment_gpt4_multilabel_RespPerc_fix.csv\nf1_weighted para AgradPerc

####LogisticRegression

In [ ]:
# CROSS VALIDATION
LogReg_pipeline = Pipeline([
                ('tfidf', vect),
                ('clf', MultiOutputClassifier(LogisticRegression(solver='sag', max_iter=1000), n_jobs=1)),
            ])


#scores = cross_val_score(LogReg_pipeline, texts, y, cv=5, scoring='f1_weighted')
scores = cross_val_score(LogReg_pipeline, texts, y, cv=KFold(n_splits=5,shuffle=True, random_state=1), scoring='f1_weighted' )

#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("CV Accuracy Scores:\n", scores)
print("Mean CV Accuracy: ", round(scores.mean(), 4))
print("Standard Deviation: ", scores.std() * 2)

''' Without Augment
CV Accuracy Scores:
 [0.42536765 0.47589809 0.64642564 0.54103129 0.63061472]
Mean CV Accuracy:  0.5439
Standard Deviation:  0.17137876330304774
'''
''' With Augment v1
CV Accuracy Scores:
 [0.49535563 0.53616199 0.62553213 0.91813192 0.70175253]
Mean CV Accuracy:  0.6554
Standard Deviation:  0.2991997467608627
'''
''' With Augment v2
CV Accuracy Scores:
 [0.50920073 0.42364637 0.56341684 0.64052527 0.71406389]
Mean CV Accuracy:  0.5702
Standard Deviation:  0.20176688324919498

augment_gpt3_multilabel
Mean CV Accuracy:  0.7091

augment_gpt3_fix
Mean CV Accuracy:  0.6507

augment_gpt4_multilabel_RespPerc.csv
Mean CV Accuracy:  0.6878

augment_gpt4_multilabel_RespPerc_fix.csv
Mean CV Accuracy:  0.6411
'''


CV Accuracy Scores:
 [0.55677582 0.60808687 0.66937264 0.64946885 0.61444366]
Mean CV Accuracy:  0.6196
Standard Deviation:  0.07733292500806675


' With Augment v2\nCV Accuracy Scores:\n [0.50920073 0.42364637 0.56341684 0.64052527 0.71406389]\nMean CV Accuracy:  0.5702\nStandard Deviation:  0.20176688324919498\n\naugment_gpt3_multilabel\nMean CV Accuracy:  0.7091\n\naugment_gpt3_fix\nMean CV Accuracy:  0.6507\n\naugment_gpt4_multilabel_RespPerc.csv\nMean CV Accuracy:  0.6878\n\naugment_gpt4_multilabel_RespPerc_fix.csv\nMean CV Accuracy:  0.6411\n'

In [ ]:
# Realizamos validación cruzada y obtenemos las predicciones
predicciones = cross_val_predict(LogReg_pipeline, texts, y, cv=5)

# Calculamos el f1_weighted para cada columna de salida y el f1_weighted medio ponderado
f1_scores = []
for i, columna in enumerate(y.columns):
    f1 = f1_score(y[columna], predicciones[:, i], average='weighted')
    f1_scores.append(f1)
    print(f"f1_weighted para {columna}: {f1:.4f}")

f1_weighted_media = sum(f1_scores) / len(f1_scores)
print(f"f1_weighted medio: {f1_weighted_media:.4f}")

''' Without Augment
f1_weighted para AgradPerc: 0.4818
f1_weighted para ApertPerc: 0.5752
f1_weighted para ExtravPerc: 0.5728
f1_weighted para NeurPerc: 0.4923
f1_weighted para RespPerc: 0.7230
f1_weighted medio: 0.5690
'''
''' With Augment v1

'''
''' With Augment v2
f1_weighted para AgradPerc: 0.4557
f1_weighted para ApertPerc: 0.6548
f1_weighted para ExtravPerc: 0.7530
f1_weighted para NeurPerc: 0.6639
f1_weighted para RespPerc: 0.6531
f1_weighted medio: 0.6361

augment_gpt3_multilabel
f1_weighted para AgradPerc: 0.5474
f1_weighted para ApertPerc: 0.6037
f1_weighted para ExtravPerc: 0.6183
f1_weighted para NeurPerc: 0.6303
f1_weighted para RespPerc: 0.7406
f1_weighted medio: 0.6281

augment_gpt3_fix
f1_weighted para AgradPerc: 0.5327
f1_weighted para ApertPerc: 0.5720
f1_weighted para ExtravPerc: 0.5837
f1_weighted para NeurPerc: 0.5177
f1_weighted para RespPerc: 0.7101
f1_weighted medio: 0.5833

augment_gpt4_multilabel_RespPerc.csv
f1_weighted para AgradPerc: 0.4272
f1_weighted para ApertPerc: 0.6455
f1_weighted para ExtravPerc: 0.7614
f1_weighted para NeurPerc: 0.6518
f1_weighted para RespPerc: 0.6429
f1_weighted medio: 0.6258

augment_gpt4_multilabel_RespPerc_fix.csv
f1_weighted para AgradPerc: 0.5333
f1_weighted para ApertPerc: 0.6565
f1_weighted para ExtravPerc: 0.7232
f1_weighted para NeurPerc: 0.7036
f1_weighted para RespPerc: 0.6751
f1_weighted medio: 0.6583

augment_gpt4_multilabel_RespPerc_fix.csv
f1_weighted para AgradPerc: 0.5333
f1_weighted para ApertPerc: 0.6565
f1_weighted para ExtravPerc: 0.7232
f1_weighted para NeurPerc: 0.7036
f1_weighted para RespPerc: 0.6751
f1_weighted medio: 0.6583
'''

f1_weighted para AgradPerc: 0.5173
f1_weighted para ApertPerc: 0.6767
f1_weighted para ExtravPerc: 0.6892
f1_weighted para NeurPerc: 0.6799
f1_weighted para RespPerc: 0.6362
f1_weighted medio: 0.6399


' With Augment v2\nf1_weighted para AgradPerc: 0.4557\nf1_weighted para ApertPerc: 0.6548\nf1_weighted para ExtravPerc: 0.7530\nf1_weighted para NeurPerc: 0.6639\nf1_weighted para RespPerc: 0.6531\nf1_weighted medio: 0.6361\n\naugment_gpt3_multilabel\nf1_weighted para AgradPerc: 0.5474\nf1_weighted para ApertPerc: 0.6037\nf1_weighted para ExtravPerc: 0.6183\nf1_weighted para NeurPerc: 0.6303\nf1_weighted para RespPerc: 0.7406\nf1_weighted medio: 0.6281\n\naugment_gpt3_fix\nf1_weighted para AgradPerc: 0.5327\nf1_weighted para ApertPerc: 0.5720\nf1_weighted para ExtravPerc: 0.5837\nf1_weighted para NeurPerc: 0.5177\nf1_weighted para RespPerc: 0.7101\nf1_weighted medio: 0.5833\n\naugment_gpt4_multilabel_RespPerc.csv\nf1_weighted para AgradPerc: 0.4272\nf1_weighted para ApertPerc: 0.6455\nf1_weighted para ExtravPerc: 0.7614\nf1_weighted para NeurPerc: 0.6518\nf1_weighted para RespPerc: 0.6429\nf1_weighted medio: 0.6258\n\naugment_gpt4_multilabel_RespPerc_fix.csv\nf1_weighted para AgradPerc